In [60]:
import pandas as pd
from scipy.stats import kendalltau

# Load the dataset
df = pd.read_csv("raw_data.csv")

pkup_plants = df[df["kingdom"] == "Plantae"]

def filter_species_with_min_days_per_site(df, min_days=10):
    """
    Filters species (and phenophases) with at least `min_days` unique first_yes_days per site.

    Parameters:
        df (pd.DataFrame): The input DataFrame containing columns:
                           'site_id', 'species_id', 'phenophase_id', 'first_yes_year', 'first_yes_day'.
        min_days (int): Minimum number of unique first_yes_days required to keep a species per site.

    Returns:
        pd.DataFrame: Filtered DataFrame containing only species/phenophases with at least `min_days` valid entries per site.
    """
    first_observed_df = df.sort_values([
        'site_id', 'species_id', 'phenophase_id', 'first_yes_year', 'first_yes_day'
    ]).drop_duplicates(subset=['site_id', 'species_id', 'phenophase_id', 'first_yes_year'], keep='first')

    valid_counts = first_observed_df.groupby(['site_id', 'species_id', 'phenophase_id'])['first_yes_day'].nunique()
    valid_species = valid_counts[valid_counts >= min_days].index

    filtered_df = first_observed_df[
        first_observed_df.set_index(['site_id', 'species_id', 'phenophase_id']).index.isin(valid_species)
    ]
    
    return filtered_df.reset_index(drop=True)

def find_species_phenophase_pairs_per_site(data):
    """
    Finds all species pairs with all possible phenophase pairs within each site.
    Includes lists of all `first_yes_day` values for matching years in a flattened DataFrame.

    Parameters:
        data (DataFrame): The filtered dataset containing columns for site_id, species_id, phenophase_id,
                          first_yes_day, and first_yes_year.

    Returns:
        DataFrame: Contains columns for site_id, species pairs, phenophase pairs, matching years,
                   and lists of first_yes_days for both species.
    """
    data = data.sort_values('first_yes_day').drop_duplicates(
        subset=['site_id', 'species_id', 'phenophase_id', 'first_yes_year']
    )

    pairs = []

    grouped_data = data.groupby(['site_id', 'species_id', 'phenophase_id'])
    for (site_a, species_a, phenophase_a), group_a in grouped_data:
        for (site_b, species_b, phenophase_b), group_b in grouped_data:
            if site_a == site_b and (species_a, phenophase_a) < (species_b, phenophase_b) and species_a != species_b:
                years_a = set(group_a['first_yes_year'])
                years_b = set(group_b['first_yes_year'])

                if years_a == years_b:
                    first_yes_days_a = [
                        group_a[group_a['first_yes_year'] == year]['first_yes_day'].values[0]
                        for year in sorted(years_a)
                    ]
                    first_yes_days_b = [
                        group_b[group_b['first_yes_year'] == year]['first_yes_day'].values[0]
                        for year in sorted(years_b)
                    ]

                    pairs.append({
                        'site_id': site_a,
                        'species_a': species_a,
                        'phenophase_a': phenophase_a,
                        'species_b': species_b,
                        'phenophase_b': phenophase_b,
                        'matching_years': list(sorted(years_a)),
                        'first_yes_days_a': first_yes_days_a,
                        'first_yes_days_b': first_yes_days_b,
                    })

    return pd.DataFrame(pairs)

def calculate_kendall_tau_per_site(df):
    """
    Calculates the Kendall tau correlation coefficient and p-value for each row in the DataFrame,
    based on the `first_yes_days_a` and `first_yes_days_b` lists, grouped by site.

    Parameters:
        df (DataFrame): A DataFrame containing `first_yes_days_a` and `first_yes_days_b` columns.

    Returns:
        DataFrame: A DataFrame with columns:
                   - site_id
                   - species_a
                   - phenophase_a
                   - species_b
                   - phenophase_b
                   - number_of_observations
                   - kendall_tau
                   - p_value
    """
    results = []

    for _, row in df.iterrows():
        days_a = row['first_yes_days_a']
        days_b = row['first_yes_days_b']

        if len(days_a) > 0 and len(days_a) == len(days_b):
            tau, p_value = kendalltau(days_a, days_b)
        else:
            tau, p_value = None, None

        results.append({
            'site_id': row['site_id'],
            'species_a': row['species_a'],
            'phenophase_a': row['phenophase_a'],
            'species_b': row['species_b'],
            'phenophase_b': row['phenophase_b'],
            'number_of_observations': len(days_a),
            'kendall_tau': tau,
            'p_value': p_value
        })

    return pd.DataFrame(results)

# Apply the functions
filtered_pkup_plants = filter_species_with_min_days_per_site(pkup_plants, min_days=10)
pairs_list_df = find_species_phenophase_pairs_per_site(filtered_pkup_plants)
results = calculate_kendall_tau_per_site(pairs_list_df)
results.to_csv("kendall_tau_results")
results

,site_id,species_a,phenophase_a,species_b,phenophase_b,number_of_observations,kendall_tau,p_value
0,8182,3,371,28,371,14,0.205718,0.319392
1,8182,3,371,28,467,14,0.000000,1.000000
2,8182,3,371,28,483,14,0.079551,0.699012
3,8182,3,371,93,371,14,0.057471,0.781787
4,8182,3,371,93,483,14,0.091954,0.657631
...,...,...,...,...,...,...,...,...
1291,17582,61,467,1212,467,10,0.022222,1.000000
1292,17582,61,483,82,483,11,0.462963,0.050225
1293,17582,61,483,1212,483,11,0.537037,0.023127
1294,17582,82,467,1212,467,10,0.022222,1.000000


In [61]:
results['site_id'].unique()

array([ 8182,  8806,  8836,  8899,  8901,  8902,  8903,  8904,  8981,
        9338,  9339,  9340,  9341,  9342,  9343,  9344, 11895, 11896,
       11897, 11899, 11999, 12002, 12003, 17582])

In [62]:
df['site_id'].unique()

array([ 8180,  8181,  8182,  8408,  8409,  8806,  8836,  8897,  8898,
        8899,  8901,  8902,  8903,  8904,  8981,  9338,  9339,  9340,
        9341,  9342,  9343,  9344, 11837, 11895, 11896, 11897, 11899,
       11999, 12002, 12003, 16258, 17582, 24412, 34051, 48788])

In [63]:
specific_pair = results[
    (results['site_id'] == 17582) &
    (results['species_a'] == 82) & (results['phenophase_a'] == 467) &
    (results['species_b'] == 1212) & (results['phenophase_b'] == 483)
]
specific_pair

,site_id,species_a,phenophase_a,species_b,phenophase_b,number_of_observations,kendall_tau,p_value


In [64]:
pairs_list_df

,site_id,species_a,phenophase_a,species_b,phenophase_b,matching_years,first_yes_days_a,first_yes_days_b
0,8182,3,371,28,371,"[2011, 2012, 2013, 2014, 2015, 2016, 2017, 201...","[4, 3, 9, 21, 13, 27, 27, 18, 1, 12, 4, 3, 21,...","[4, 3, 24, 16, 13, 11, 19, 18, 7, 10, 10, 17, ..."
1,8182,3,371,28,467,"[2011, 2012, 2013, 2014, 2015, 2016, 2017, 201...","[4, 3, 9, 21, 13, 27, 27, 18, 1, 12, 4, 3, 21,...","[11, 10, 24, 23, 16, 13, 19, 2, 13, 17, 19, 17..."
2,8182,3,371,28,483,"[2011, 2012, 2013, 2014, 2015, 2016, 2017, 201...","[4, 3, 9, 21, 13, 27, 27, 18, 1, 12, 4, 3, 21,...","[11, 3, 24, 23, 16, 13, 19, 2, 13, 17, 14, 17,..."
3,8182,3,371,93,371,"[2011, 2012, 2013, 2014, 2015, 2016, 2017, 201...","[4, 3, 9, 21, 13, 27, 27, 18, 1, 12, 4, 3, 21,...","[4, 30, 14, 6, 6, 3, 8, 5, 1, 19, 5, 4, 9, 3]"
4,8182,3,371,93,483,"[2011, 2012, 2013, 2014, 2015, 2016, 2017, 201...","[4, 3, 9, 21, 13, 27, 27, 18, 1, 12, 4, 3, 21,...","[2, 5, 21, 9, 11, 6, 14, 14, 1, 27, 21, 11, 9, 8]"
...,...,...,...,...,...,...,...,...
1291,17582,61,467,1212,467,"[2015, 2016, 2017, 2018, 2019, 2020, 2021, 202...","[13, 14, 9, 16, 17, 7, 6, 11, 10, 12]","[28, 6, 9, 1, 17, 7, 11, 19, 10, 18]"
1292,17582,61,483,82,483,"[2014, 2015, 2016, 2017, 2018, 2019, 2020, 202...","[30, 13, 6, 9, 16, 17, 7, 11, 11, 10, 12]","[16, 13, 6, 9, 16, 17, 7, 28, 1, 8, 30]"
1293,17582,61,483,1212,483,"[2014, 2015, 2016, 2017, 2018, 2019, 2020, 202...","[30, 13, 6, 9, 16, 17, 7, 11, 11, 10, 12]","[30, 13, 1, 9, 1, 17, 7, 11, 19, 10, 18]"
1294,17582,82,467,1212,467,"[2015, 2016, 2017, 2018, 2019, 2020, 2021, 202...","[13, 6, 9, 16, 17, 7, 2, 1, 8, 12]","[28, 6, 9, 1, 17, 7, 11, 19, 10, 18]"


In [65]:
specific_pair = pairs_list_df[
    (pairs_list_df['site_id'] == 17582) &
    (pairs_list_df['species_a'] == 82) & (pairs_list_df['phenophase_a'] == 467) &
    (pairs_list_df['species_b'] == 1212) & (results['phenophase_b'] == 483)
]
specific_pair

,site_id,species_a,phenophase_a,species_b,phenophase_b,matching_years,first_yes_days_a,first_yes_days_b


In [66]:
df["site_id"].unique()


array([ 8180,  8181,  8182,  8408,  8409,  8806,  8836,  8897,  8898,
        8899,  8901,  8902,  8903,  8904,  8981,  9338,  9339,  9340,
        9341,  9342,  9343,  9344, 11837, 11895, 11896, 11897, 11899,
       11999, 12002, 12003, 16258, 17582, 24412, 34051, 48788])

In [67]:
pairs_list_df.to_csv("valid_pairs")

In [68]:
pkup_plants['species_id'].unique()

array([ 444, 1174,   93,   28,  102,    3, 1172,  823, 1065,   12, 1187,
         61,   98,   81,   82,  100, 1192, 2191,   79,  970,    2, 1201,
         60,   33, 1159,  941,   75,  949, 1184,   97,   91, 1199, 1189,
       1177,   68, 1190,   80, 1019, 1185,   35, 1176,   76, 1181, 1016,
       2114, 1175, 1041, 1039, 2177,  782,  724, 1212, 1179,  821, 2131,
          7, 2197,   67,   74, 2195,   95])

In [69]:
results[(results['species_a'] == 1177) | (results['species_b'] == 1177)]

,site_id,species_a,phenophase_a,species_b,phenophase_b,number_of_observations,kendall_tau,p_value
424,8902,3,371,1177,371,11,0.129630,0.583519
445,8902,3,500,1177,371,11,-0.018519,0.937572
468,8902,79,371,1177,371,11,0.092593,0.695348
474,8902,102,371,1177,371,11,0.293590,0.211522
1225,11896,3,483,1177,483,12,0.359375,0.111389
1229,11896,100,483,1177,483,12,0.124035,0.580639
1231,11896,941,483,1177,483,12,0.515625,0.022363
1241,12002,941,483,1177,483,12,0.198479,0.371565
1242,12002,941,498,1177,498,11,0.330289,0.159854
1244,12002,1159,467,1177,467,11,0.351852,0.136721


In [70]:
new = pd.read_csv("raw_data.csv")
new["species_id"].unique()


array([ 444, 1174,   93,   28,  245, 1226,  102,    3, 1172, 1227,  823,
       1233, 1126, 1098, 1065,   12, 1236, 1090, 1230,  377, 1187,   61,
         98,   81,   82,  100, 1192, 2191,   79,  970,    2, 1201,   60,
         33, 1159,  941,   75,  949, 1184,   97,   91, 1199, 1189, 1177,
         68, 1190,   80, 1019, 1185,   35, 1176,   76, 1181, 1016, 2114,
       1175, 1041, 1039, 2177,  782,  724, 1212, 1179,  821, 2131,    7,
       2197,   67,   74, 2195,   95])

In [71]:
print(results['site_id'].nunique())
total_unique_species = pd.concat([results['species_a'], results['species_b']]).unique()
print(f"Total unique species: {len(total_unique_species)}")

24
Total unique species: 36


In [72]:
stat_sig_results = results[results["p_value"] <= 0.05]
stat_sig_results.to_csv("stat_sig_kendall_tau_results")